In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Flatten, Dense

img_width, img_height = 100, 100

train_data_dir = 'dataNewV2/training'

validation_data_dir = 'dataNewV2/validation'

In [0]:
train_samples = 6600
import keras
validation_samples = 1200
test_samples=600

epoch = 1
vgg16_model=keras.applications.vgg16.VGG16()

In [0]:
#VGG16 MODEL
vgg16_model.summary()
type(vgg16_model)
from keras import applications
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
base_model.trainable = True

for layer in base_model.layers:
    if layer.name == 'block5_conv1':
        layer.trainable = True
    else:
        layer.trainable = False

In [0]:
model=Sequential()
model.add(base_model)

model.add(Flatten())
model.add(Dense(256, activation='relu'))

model.add(Dense(6, activation='softmax'))
for layer in model.layers[:10]:
    layer.trainable = True
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
fid = drive.ListFile({'q':"title='dataNewV2.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('dataNewV2.zip')

In [0]:
from google.colab import files
files.os.listdir()

In [0]:
import zipfile
with zipfile.ZipFile("dataNewV2.zip","r") as zip_ref:
    zip_ref.extractall()

In [0]:
files.os.listdir()

In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=10,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=10,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        samples_per_epoch=train_samples,
        nb_epoch=10,
        validation_data=validation_generator,
        nb_val_samples=validation_samples)

model.save_weights('neuralnet.h5')

In [0]:
from keras.applications import ResNet50, Xception, InceptionResNetV2

In [0]:
#XCEPTION MODEL
DIMS = (100,100,3)
model = Xception(include_top=False,
                     weights='imagenet',
                     input_shape=(100, 100, 3),
                     pooling='avg')

In [0]:
print(model.summary())

In [0]:
modelv=model
modelv.trainable = True

for layer in modelv.layers:
        layer.trainable = True


In [0]:

modelq=Sequential()
modelq.add(modelv)


modelq.add(Dense(256, activation='relu'))

modelq.add(Dense(6, activation='softmax'))

modelq.summary()
modelq.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
             metrics=['accuracy'])

In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=10,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=10,
        class_mode='categorical')

history=modelq.fit_generator(
        train_generator,
        samples_per_epoch=train_samples,
        nb_epoch=20,     
        validation_data=validation_generator,
        nb_val_samples=validation_samples)


In [0]:
test_generator = test_datagen.flow_from_directory(
        'dataNewV2/testing',
        target_size=(100, 100),
        batch_size=1,
        class_mode = None,  # only data, no labels
        shuffle = False)  # keep data in same order as labels
probabilities = modelq.predict_generator(test_generator, 600)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np


y_pred = np.asarray(probabilities)
y_pred = np.argmax(probabilities,axis=1)
print(y_pred)
y_true = np.array([0] * 100 + [1] * 100 + [2] * 100 + [3]*100 + [4]*100+[5]*100)
print(accuracy_score(y_true, y_pred))

In [0]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

In [0]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['validation', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()